In [8]:
nbAppear = dict()

def lireDict(dictionnaire,clef):
    if (clef in dictionnaire.keys()):
        return dictionnaire[clef]
    return None


def ajout(c1,c2):
    premiereLettre = lireDict(nbAppear,c1)
    if premiereLettre != None :
        secondeLettre = lireDict(premiereLettre,c2)
        if secondeLettre != None :
            nbAppear[c1][c2] += 1
        else :
            nbAppear[c1][c2] = 1
    else :
        nbAppear[c1] = {c2 : 1}
        
def lit(mot):
    for i in range(mot-1):
        ajout(mot[i],mot[i+1])

def estLettre(c):
    return True if c in [chr(c) for c in range(65,91)]+[chr(c) for c in range(97,123)] else False

def lit2(texte,i):
    mot = ""
    while estLettre(texte[i]):
        mot+=texte[i]
        i+=1
    return (mot,i+1)

def mat():
    fichier = open("./ProjetPython/notredame.txt", "r")
    text = fichier.read().split('\n')
    i = 0
    for ligne in text:
        motlit2(ligne,i)